In [2]:
from agent_framework.human_input import CLIHumanHandler, AskHumanTool
from rich.pretty import pprint

from agent_framework.tools.builtin_tools import GetCurrentTimeTool, CalculatorTool
from agent_framework.agents.react_agent import ReActAgent
from agent_framework.model_clients import OpenAIClient

handler = CLIHumanHandler()

# 2. Create the AskHuman tool (max 10 questions per run)
ask_tool = AskHumanTool(
    handler=handler,
    max_requests_per_run=10,
)

# 3. Set up the agent with HITL support
client = OpenAIClient(model="gpt-5-mini")

agent = ReActAgent(
    name="hitl-assistant",
    description="An assistant that asks for human input when needed",
    model_client=client,
    tools=[ask_tool, CalculatorTool(), GetCurrentTimeTool()],
    system_instructions="""
You are a helpful AI assistant. Your PRIMARY strategy for planning, 
decision-making, or preference-based questions is to ASK THE USER FIRST 
using the ask_human tool. Do NOT try to decide alone.

ALWAYS use ask_human when:
- Planning or organizing (events, schedules, budgets, etc)
- Choosing between multiple valid options
- The user's preferences would affect the recommendation
- You're uncertain which approach they'd prefer

Guidelines:
- Present 2-3 distinct, actionable options
- Provide brief context explaining WHY you're asking
- Users can select an option OR type "Other" with their own answer
- After getting their input, incorporate it fully into your plan
- You can ask up to 10 questions per conversation

Example: 
  User: "Plan a team dinner"
  You: "Great! Let me ask a few preferences first..."
  Then: Use ask_human to ask about cuisine, budget, date, etc.

Be conversational and explain your thinking.""",
    max_iterations=10,
)
result = await agent.run(
    "I need to plan a team building event for my startup. "
    "We have 12-15 people, a budget of $2000, and 2 weeks to plan. "
    "What's the best approach?"
    )

{"timestamp": "2026-02-14T13:57:57.596509", "level": "INFO", "name": "agent_framework", "message": "[hitl-assistant] Starting run: I need to plan a team building event for my startup. We have 12-15 people, a bud..."}
{"timestamp": "2026-02-14T13:58:03.486430", "level": "INFO", "name": "httpx", "message": "HTTP Request: POST https://api.openai.com/v1/responses \"HTTP/1.1 200 OK\""}
{"timestamp": "2026-02-14T13:58:03.655311", "level": "INFO", "name": "agent_framework", "message": "[hitl-assistant] Step 1: tool calls → ['ask_human']"}
{"timestamp": "2026-02-14T13:58:03.656741", "level": "INFO", "name": "agent_framework", "message": "[hitl-assistant] Executing ask_human({'question': 'Which type of team-building event would you prefer for your 12–15 person startup, given the $2,000 budget and 2-week planning window?', 'context': 'Different formats fit different goals, logistics, and costs. Pick the approach you prefer and I’ll build a detailed plan (venue, schedule, vendors, budget breakdow

In [3]:
# Display the result
print("\n📋 AGENT RESPONSE:")
print("-" * 60)
print(result.summary())

# Show all human interactions that occurred
history = ask_tool.interaction_history
if history:
    print("\n" + "="*60)
    print(f"💬 HUMAN INTERACTIONS ({len(history)} total)")
    print("="*60)
    for i, interaction in enumerate(history, 1):
        print(f"\n[Interaction {i}]")
        print(f"  Question: {interaction['question']}")
        print(f"  Options: {', '.join([f'{i}={o}' for i, o in enumerate(interaction['options'], 1)])}")
        print(f"  User chose: '{interaction['answer']}'")
        print(f"             (freeform={interaction['is_freeform']})")
else:
    print("\n⚠️  No human interactions recorded (agent may have skipped asking)")

print("\n" + "="*60)
print("✅ Demo Complete!")
print("="*60)


📋 AGENT RESPONSE:
------------------------------------------------------------
[completed] hitl-assistant | 4/10 steps | 3 tool calls (ask_humanx3) | 8640 tokens | 52.90s

💬 HUMAN INTERACTIONS (3 total)

[Interaction 1]
  Question: Which type of team-building event would you prefer for your 12–15 person startup, given the $2,000 budget and 2-week planning window?
  Options: 1=Structured half-day onsite/nearby workshop with a professional facilitator, interactive exercises, and catered lunch (focus: communication and collaboration)., 2=Evening offsite social + activity (escape room or cooking class) with group dinner — fun and bonding with lower time commitment., 3=Outdoor adventure day (low ropes course, guided hike, or kayaking) with picnic — high engagement and experiential bonding. (Note: may require accessibility considerations).
  User chose: 'Structured half-day onsite/nearby workshop with a professional facilitator, interactive exercises, and catered lunch (focus: communication

In [4]:
pprint(result)

AgentRunResult(
│   run_id='df165cfb-1c7f-4a00-b54f-9d0107be3928',
│   agent_name='hitl-assistant',
│   output=[
│   │   'Great — thanks for the choices. Based on your selection (structured half-day onsite workshop, morning 9:00–12:30 at your office, 12 attendees, no special requirements), here’s a ready-to-run plan you can execute in the next two weeks within your $2,000 budget.\n\nSummary plan (one-line)\n- Half-day facilitated communication & collaboration workshop onsite, catered breakfast + light lunch, AV + materials, professional facilitator — total ~ $1,760–$1,900 (leaving a buffer).\n\nHigh-level objectives\n- Improve cross-functional communication\n- Practice collaborative problem solving\n- Create 2–3 concrete team commitments to improve workflows\n\nRecommended agenda (9:00–12:30)\n- 9:00–9:15 Arrival, coffee/tea, welcome, purpose & outcomes\n- 9:15–9:30 Quick icebreaker (speed intros + one work challenge)\n- 9:30–10:15 Core module 1: Communication fundamentals + active listening practice (interactive)\n- 10:15–10:30 Break (refreshments)\n- 10:30–11:15 Collaborative problem-solving activity (Marshmallow Challenge or short design challenge in mixed teams)\n- 11:15–11:45 Role clarity & strengths mapping (small-group share)\n- 11:45–12:10 Action planning: team commitments, owners, timelines\n- 12:10–12:25 Feedback & close (5-min survey)\n- 12:25–12:30 Quick wrap and next steps\n\nRecommended activities (why they work)\n- Marshmallow Challenge (fast, cheap, reveals communication + planning issues)\n- Active listening paired exercises (practical, immediate skill practice)\n- Strengths/role mapping + 1-2 min shares (builds role clarity)\n- Action planning with named owners (ensures outcomes)\n\nBudget breakdown (estimates; total ~ $1,760)\n- Facilitator (half-day + prep): $1,000\n  - Note: local trainers vary; this is a mid-range estimate for a 3–4 hour onsite session with prep.\n- Catering (breakfast + light lunch for 12): $300 ($25/person)\n  - Example: breakfast pastries, fruit, coffee/tea; boxed sandwiches or buffet light lunch\n- AV/equipment rental (projector, clicker, speaker if needed): $150\n- Materials & prints (flipchart, markers, printed worksheets, name tags): $100\n- Facilitator travel/incidentals: $50\n- Contingency / small extras (swag, extra refreshments): $160\nTotal: $1,760 — leaves ~$240 available if facilitator quotes come higher or you want small swag or upgrade catering.\n\nIf you want to save money: hire an internal/peer facilitator and allocate ~$400 for a facilitator kit + increased catering (I can provide a facilitator script and materials).\n\nTwo-week planning timeline (task-by-task)\n- Day 14 (Today)\n  - Confirm date, time, headcount (done).\n  - Define 2–3 workshop goals (see objectives above).\n  - Book the budget (ok to proceed up to $2,000).\n\n- Days 13–10 (Immediate vendor sourcing & selection)\n  - Request 3 facilitator quotes (LinkedIn local trainers, corporate trainers, event marketplaces).\n  - Request 2–3 catering quotes for 12 people.\n  - Check in-house AV and room capacity; if missing, get rental quote.\n  - Decide whether to hire pro facilitator or use internal.\n\n- Days 9–7 (Confirm vendors & agenda)\n  - Book facilitator (deposit if required).\n  - Finalize agenda with facilitator; share objectives and sample program.\n  - Confirm catering & dietary needs (none currently).\n  - Order any materials (flipcharts, props).\n\n- Days 6–4 (Communications & logistics)\n  - Send official calendar invite and RSVP email (sample below).\n  - Prepare seating plan (mixed teams).\n  - Arrange room layout, AV test time.\n  - Create printed materials and name tags.\n\n- Days 3–1 (Confirmations & final prep)\n  - Confirm final headcount with catering (cutoff 48–72 hours).\n  - Confirm facilitator arrival time and run-of-show.\n  - Print handouts, prepare materials kit, charge devices.\n  - Assign an onsite host (10–15 min before for setup & vendor sign-in).\n\n- Event day\n  - Set up 60–90 

In [7]:
from IPython.display import display, Markdown
display(Markdown(result.output[0]))

Great — thanks for the choices. Based on your selection (structured half-day onsite workshop, morning 9:00–12:30 at your office, 12 attendees, no special requirements), here’s a ready-to-run plan you can execute in the next two weeks within your $2,000 budget.

Summary plan (one-line)
- Half-day facilitated communication & collaboration workshop onsite, catered breakfast + light lunch, AV + materials, professional facilitator — total ~ $1,760–$1,900 (leaving a buffer).

High-level objectives
- Improve cross-functional communication
- Practice collaborative problem solving
- Create 2–3 concrete team commitments to improve workflows

Recommended agenda (9:00–12:30)
- 9:00–9:15 Arrival, coffee/tea, welcome, purpose & outcomes
- 9:15–9:30 Quick icebreaker (speed intros + one work challenge)
- 9:30–10:15 Core module 1: Communication fundamentals + active listening practice (interactive)
- 10:15–10:30 Break (refreshments)
- 10:30–11:15 Collaborative problem-solving activity (Marshmallow Challenge or short design challenge in mixed teams)
- 11:15–11:45 Role clarity & strengths mapping (small-group share)
- 11:45–12:10 Action planning: team commitments, owners, timelines
- 12:10–12:25 Feedback & close (5-min survey)
- 12:25–12:30 Quick wrap and next steps

Recommended activities (why they work)
- Marshmallow Challenge (fast, cheap, reveals communication + planning issues)
- Active listening paired exercises (practical, immediate skill practice)
- Strengths/role mapping + 1-2 min shares (builds role clarity)
- Action planning with named owners (ensures outcomes)

Budget breakdown (estimates; total ~ $1,760)
- Facilitator (half-day + prep): $1,000
  - Note: local trainers vary; this is a mid-range estimate for a 3–4 hour onsite session with prep.
- Catering (breakfast + light lunch for 12): $300 ($25/person)
  - Example: breakfast pastries, fruit, coffee/tea; boxed sandwiches or buffet light lunch
- AV/equipment rental (projector, clicker, speaker if needed): $150
- Materials & prints (flipchart, markers, printed worksheets, name tags): $100
- Facilitator travel/incidentals: $50
- Contingency / small extras (swag, extra refreshments): $160
Total: $1,760 — leaves ~$240 available if facilitator quotes come higher or you want small swag or upgrade catering.

If you want to save money: hire an internal/peer facilitator and allocate ~$400 for a facilitator kit + increased catering (I can provide a facilitator script and materials).

Two-week planning timeline (task-by-task)
- Day 14 (Today)
  - Confirm date, time, headcount (done).
  - Define 2–3 workshop goals (see objectives above).
  - Book the budget (ok to proceed up to $2,000).

- Days 13–10 (Immediate vendor sourcing & selection)
  - Request 3 facilitator quotes (LinkedIn local trainers, corporate trainers, event marketplaces).
  - Request 2–3 catering quotes for 12 people.
  - Check in-house AV and room capacity; if missing, get rental quote.
  - Decide whether to hire pro facilitator or use internal.

- Days 9–7 (Confirm vendors & agenda)
  - Book facilitator (deposit if required).
  - Finalize agenda with facilitator; share objectives and sample program.
  - Confirm catering & dietary needs (none currently).
  - Order any materials (flipcharts, props).

- Days 6–4 (Communications & logistics)
  - Send official calendar invite and RSVP email (sample below).
  - Prepare seating plan (mixed teams).
  - Arrange room layout, AV test time.
  - Create printed materials and name tags.

- Days 3–1 (Confirmations & final prep)
  - Confirm final headcount with catering (cutoff 48–72 hours).
  - Confirm facilitator arrival time and run-of-show.
  - Print handouts, prepare materials kit, charge devices.
  - Assign an onsite host (10–15 min before for setup & vendor sign-in).

- Event day
  - Set up 60–90 minutes prior; AV test.
  - Host greets facilitator and caterer; run the workshop.
  - Collect quick feedback survey.

Sample facilitator brief to send (one paragraph)
- “We’re a 12-person startup running a 3.5-hour onsite workshop focused on improving communication and collaboration. Goal: practical exercises that improve day-to-day teamwork and deliver 2–3 team commitments. Date/time: [date], 9:00–12:30. Room: office conference; AV available: projector & speaker. Please propose a half-day agenda, cost including prep, and any materials you need. Please confirm availability and references.”

Sample invite email (short)
- Subject: Team Workshop — Communication & Collaboration — [Date], 9:00–12:30
- Body: Brief purpose, agenda highlight, request to arrive at 8:50 for coffee, RSVP by [date]. Mention any prep (none required).

Quick 5-question feedback survey (post-event)
1) How useful was the session? (1–5)
2) Which exercise was most valuable?
3) What one behavior will you change?
4) Were objectives met? (Yes/No; explain)
5) Any suggestions for future sessions?

Contingency options
- If no facilitator available: run with internal facilitator using a provided 3.5-hour script (I can draft this if you want).
- If catering issues arise: potluck-style or order boxed lunches from local deli (still within budget).
- If someone can’t attend: run the same content in a shorter follow-up for them or record key takeaways.

Vendor-sourcing tips
- Ask facilitator for a 60–90 min pre-call to tailor content — include in quote.
- Get 2–3 quotes and compare: price, prep time, materials, and references.
- For catering, ask for set menus for startups/small groups; confirm delivery and setup time.
- Negotiate a small discount or add-ons (free coffee, extra pastry) for small business orders.

Ready-made deliverables I can produce next
- Draft outreach message to 3 facilitator candidates (for you to paste/send)
- Exact facilitator shortlist criteria and vendor questions you can use
- Complete run-sheet for the day (minute-by-minute)
- Full facilitator script and printable handouts if you choose internal facilitation
- Polished invite and RSVP tracking template

Which of those deliverables would you like me to prepare next? (I recommend: facilitator outreach + 1-page run-sheet)